In [1]:
import torch
from algo_reasoning.src.sampler import CLRSDataset
from pathlib import Path

/home/ronaldalbert/Documents/env/algo_reasoning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device =  torch.device("cpu")

## Load Pretrained Model

In [3]:
from algo_reasoning.src.models.network import EncodeProcessDecode
from algo_reasoning.src.lightning.AlgorithmicReasoningTask import AlgorithmicReasoningTask 
from algo_reasoning.src.specs import CLRS_30_ALGS
from algo_reasoning.src.losses.AlgorithmicReasoningLoss import AlgorithmicReasoningLoss

In [4]:
algorithms = ["dfs"]

In [5]:
ckpt_path = str(next(Path(f"../checkpoints/{algorithms[0]}/").glob(f"{algorithms[0]}-spectralmpnn0*")))
#ckpt_path = str(next(Path(f"../checkpoints/{algorithms[0]}/").glob(f"{algorithms[0]}-mpnn*")))

model = EncodeProcessDecode(algorithms, processor='spectralmpnn2')
loss_fn = AlgorithmicReasoningLoss()

model = AlgorithmicReasoningTask.load_from_checkpoint(ckpt_path, model=model, loss_fn=loss_fn, strict=False).model.to(device)

In [ ]:
# Calculating embeddings with model reg = 0.0

ckpt_path = str(next(Path(f"../checkpoints/checkpoints/{algorithms[0]}/").glob(f"{algorithms[0]}-hidden_reg=0.0*")))

model = EncodeProcessDecode(algorithms)
loss_fn = AlgorithmicReasoningLoss()

model_0 = AlgorithmicReasoningTask.load_from_checkpoint(ckpt_path, model=model, loss_fn=loss_fn).model.to(device)

In [ ]:
# Calculating embeddings with model reg = 0.0

ckpt_path = str(next(Path(f"../checkpoints/checkpoints/{algorithms[0]}/").glob(f"{algorithms[0]}-hidden_reg=0.1*")))

model = EncodeProcessDecode(algorithms)
loss_fn = AlgorithmicReasoningLoss()

model_1 = AlgorithmicReasoningTask.load_from_checkpoint(ckpt_path, model=model, loss_fn=loss_fn).model.to(device)

# Graph Fourier Analysis of Algorithmic Embeddings

## Toy Example: Graph Fourier Transform of BFS Embeddings

In [6]:

def fourier_transform(embeddings, adj_matrix):
    """
    Compute the Fourier Transform of the embeddings.

    Args:
        embeddings: a tensor of shape (batch_size, sequence_length, embedding_dim)

    Returns:
        a tensor of shape (batch_size, sequence_length, embedding_dim)
    """
    degrees = torch.sum(adj_matrix, dim=1)
    degree_matrix = torch.stack([torch.diag(degrees[d]) for d in range(degrees.size(0))], dim=0)
    laplacian = degree_matrix - adj_matrix
    
    result = torch.linalg.eigh(laplacian)
    eigenvalues = result.eigenvalues
    eigenvectors = result.eigenvectors
    
    eigenvalues[torch.isclose(eigenvalues, torch.tensor(0.))] = 0.

    return eigenvectors.transpose(-2, -1)@embeddings, eigenvectors, eigenvalues

In [7]:
def mean_average_distance(x):
    """
    Compute the mean average distance of the x.

    Args:
        x: a tensor of shape (batch_size, sequence_length, embedding_dim)

    Returns:
        a tensor of shape (batch_size, sequence_length, embedding_dim)
    """

    embeddings_sim = torch.matmul(x, x.transpose(-2, -1))/(torch.norm(x, dim=-1).unsqueeze(-1)*torch.norm(x, dim=-1).unsqueeze(-2))
    distance = 1 - embeddings_sim

    mad = torch.mean(torch.mean(distance, dim=-1), dim=-1)

    return mad

In [31]:
def dirichlet_energy(x, adj_matrix=None):
    """
    Computes the Dirichlet Energy of the node Embeddings as defined by: https://arxiv.org/abs/2303.10993

    Args:
        x: a tensor of shape (batch_size, sequence_length, embedding_dim)
        adj_matrix: binary tensor of the graph adjacency matrix (sequence_length, sequence_length)

    Returns:
        a tensor of shape (batch_size, sequence_length)
    """
    batch_size, nb_nodes, embedding_dim = x.size()
    
    if adj_matrix is None:
        adj_matrix = torch.ones(batch_size, nb_nodes, nb_nodes, device=device)

    degrees = torch.sum(adj_matrix, dim=1)
    energy = 0
    for i in range(nb_nodes):
        node_diff = (x[:, i]/degrees[:,i].unsqueeze(-1)).unsqueeze(1) - x/degrees.unsqueeze(-1)
        for j in range(nb_nodes):
            differences = node_diff[adj_matrix[:, i].bool()]
            diff_norms = torch.norm(differences, dim=1)
            energy += torch.sum(diff_norms)

    return torch.sqrt(energy/batch_size)

In [ ]:
# Load dataset
algorithm_args = {
    'p': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
}

nb_nodes = 16
ds = CLRSDataset(algorithms, nb_nodes, 32, 1000, seed=7, algorithms_args=algorithm_args)

In [ ]:
obj = next(iter(ds)).to(device=device)

In [ ]:
obj.max_length

In [ ]:
output = model(obj)
embeddings = output.hidden_embeddings
embeddings.shape

In [ ]:
fourier_embeddings, eigenvectors, eigenvalues = fourier_transform(embeddings[:, -4], adj_matrix)

In [ ]:
eigenvectors@(torch.diag_embed(eigenvalues)@(eigenvectors.transpose(-1, -2)@embeddings[:, -4])) 

In [ ]:
import matplotlib.pyplot as plt

plot_data = torch.mean(fourier_embeddings, dim=-1)

plt.bar(x=[i for i in range(nb_nodes)], height=plot_data[11].detach().cpu().numpy())

In [ ]:
import matplotlib.pyplot as plt

mad = [torch.mean(dirichlet_energy(embeddings[:, i, :, :].squeeze()), dim=0).item() for i in range(embeddings.size(1))]
plt.plot(mad)

In [ ]:
mad = [torch.mean(dirichlet_energy(embeddings[:, i, :, :].squeeze(), adj_matrix), dim=0).item() for i in range(embeddings.size(1))]
plt.plot(mad)

In [ ]:
import matplotlib.pyplot as plt
difference_over_length = torch.mean(torch.abs(plot_data[:, :, 0] - torch.mean(plot_data[:, :, 1:])), dim=0)

plt.plot(difference_over_length.detach().cpu().numpy())

In [ ]:
laplacian = torch.diag(torch.ones(nb_nodes, device=device)*nb_nodes) - torch.ones((nb_nodes, nb_nodes), device=device)
_embeddings = embeddings.clone()[0, :, :]
variability = torch.sum((_embeddings.transpose(1, 2)@laplacian)*_embeddings.transpose(1,2), dim=2)

plt.plot(torch.mean(variability, dim=1).detach().cpu().numpy())

In [ ]:
def _mean_corr_coef(embeddings):
    mean_corr_coef = None
    for i in range(embeddings.size(0)):
        if mean_corr_coef is None:
            mean_corr_coef = embeddings[i, 15].corrcoef()
        else:
            mean_corr_coef += embeddings[i, 15].corrcoef()

    mean_corr_coef = mean_corr_coef/embeddings.size(0)

    return mean_corr_coef

In [ ]:
import pandas as pd

def calculate_mean_corr_coef(algos, model):
    mean_corr = []
    
    for algo in CLRS_30_ALGS:
        nb_nodes = 16
        ds = CLRSDataset(algorithms, nb_nodes, 8, 1000, seed=7, algorithms_args=algorithm_args)
    
        obj = next(iter(ds)).to(device=device)
    
        output = model(obj)
        embeddings = output.hidden_embeddings
        
        mean_corr.append(_mean_corr_coef(embeddings).mean())

    return mean_corr

mean_corr_coef = calculate_mean_corr_coef(CLRS_30_ALGS, model)
mean_corr_coef_0 = calculate_mean_corr_coef(CLRS_30_ALGS, model_0)
mean_corr_coef_1 = calculate_mean_corr_coef(CLRS_30_ALGS, model_1)

In [ ]:
mean_corr_df = pd.DataFrame({"algoritmhs": CLRS_30_ALGS, 
                             "mean_corr_0.5":mean_corr_coef, 
                             "mean_corr_0.1":mean_corr_coef_1,
                             "mean_corr_0.0":mean_corr_coef_0})

mean_corr_df

# Low OOD Dataset Analysis

In [32]:
import yaml


def load_algorithm_args(args_file):
    with open(args_file, 'r') as f:
        args = yaml.safe_load(f)

    return args

algorithm_args = load_algorithm_args("../algorithm_args/default.yaml")

In [33]:
low_ood_algorithms = ["articulation_points", "activity_selector", "binary_search", "bubble_sort", "dfs",
                     "find_maximum_subarray_kadane", "floyd_warshall", "heapsort", "insertion_sort", "kmp_matcher",
                     "matrix_chain_order", "mst_kruskal", "mst_prim", "naive_string_matcher", "quickselect",
                     "quicksort", "strongly_connected_components", "task_scheduling", "topological_sort"]

nb_nodes = 16

dirichlet_en = dict()
for algorithm in low_ood_algorithms:
    print(algorithm)
    ckpt_path = str(next(Path(f"../_checkpoints/{algorithm}/").glob(f"{algorithm}-hidden_reg=0.0*")))
    model = EncodeProcessDecode([algorithm], processor='mpnn')

    loss_fn = AlgorithmicReasoningLoss()

    model = AlgorithmicReasoningTask.load_from_checkpoint(ckpt_path, model=model, loss_fn=loss_fn, strict=False).model.to(device)


    ds = CLRSDataset([algorithm], nb_nodes, 8, 1000, seed=7, algorithms_args=algorithm_args)
    obj = next(iter(ds)).to(device=device)
    
    output = model(obj)
    embeddings = output.hidden_embeddings

    dirichlet_en[algorithm] = [dirichlet_energy(embeddings[:, i, :, :].squeeze()).item() for i in range(embeddings.size(1))]

articulation_points
activity_selector
binary_search
bubble_sort
dfs
find_maximum_subarray_kadane
floyd_warshall
heapsort
insertion_sort
kmp_matcher
matrix_chain_order
mst_kruskal
mst_prim
naive_string_matcher
quickselect
quicksort
strongly_connected_components
task_scheduling
topological_sort


In [34]:
dirichlet_avg = []
for key in dirichlet_en.keys():
    dirichlet_avg.append(torch.tensor(dirichlet_en[key]).mean().item())
dirichlet_avg

[43.80147933959961,
 42.86289596557617,
 53.79662322998047,
 44.582603454589844,
 49.861019134521484,
 51.77833557128906,
 47.81977081298828,
 41.594974517822266,
 45.00823211669922,
 63.96101760864258,
 52.29963684082031,
 35.246368408203125,
 42.25633239746094,
 64.68964385986328,
 38.785301208496094,
 43.29964065551758,
 45.81415939331055,
 51.948089599609375,
 49.73229217529297]

In [35]:
from algo_reasoning.src.specs import CLRS_30_ALGS

high_ood_algorithms = [alg for alg in CLRS_30_ALGS if alg not in low_ood_algorithms]

high_ood_en = dict()

for algorithm in high_ood_algorithms:
    print(algorithm)
    ckpt_path = str(next(Path(f"../_checkpoints/{algorithm}/").glob(f"{algorithm}-hidden_reg=0.0*")))
    model = EncodeProcessDecode([algorithm], processor='mpnn')

    loss_fn = AlgorithmicReasoningLoss()

    model = AlgorithmicReasoningTask.load_from_checkpoint(ckpt_path, model=model, loss_fn=loss_fn, strict=False).model.to(device)


    ds = CLRSDataset([algorithm], nb_nodes, 8, 1000, seed=7, algorithms_args=algorithm_args)
    obj = next(iter(ds)).to(device=device)
    
    output = model(obj)
    embeddings = output.hidden_embeddings

    high_ood_en[algorithm] = [dirichlet_energy(embeddings[:, i, :, :].squeeze()).item() for i in range(embeddings.size(1))]

bellman_ford
bfs
bridges
dag_shortest_paths
dijkstra
graham_scan
jarvis_march
lcs_length
minimum
optimal_bst
segments_intersect


In [38]:
dirichlet_avg = []
for key in high_ood_en.keys():
    dirichlet_avg.append(torch.tensor(high_ood_en[key]).mean().item())
dirichlet_avg

[41.158023834228516,
 38.2791748046875,
 44.376800537109375,
 44.11208724975586,
 45.68049621582031,
 54.25910186767578,
 51.69864273071289,
 56.80711364746094,
 43.57819366455078,
 45.41507339477539,
 13.613958358764648]